In [1]:
import numpy as np
import logging
from gensim.models import Word2Vec
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [2]:
def get_data(path):
    texts = []
    labels = []
    read_file = open(path, 'r')
    for line in read_file:
        line = line.replace('\n','')
        items = line.split('\t')
        texts.append(items[0])
        labels.append(items[1])
    read_file.close()

    return texts, labels

In [3]:
path = 'POS_Sanskrit.txt'

In [4]:
words, tags = get_data(path)

In [5]:
words = [words]
labels = set(tags)

In [6]:
model_wv = Word2Vec(words, size=300, window=1, min_count=1, workers=4)

In [7]:
m,n = np.shape(words)
X = []
for i in range(n):
    X.append(model_wv.wv[words[0][i]])

In [8]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(tags)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [10]:
print(np.shape(X_train),np.shape(y_train))
print(np.shape(X_test), np.shape(y_test))

(29997, 300) (29997,)
(12856, 300) (12856,)


In [11]:
#clf = SVC()
#clf = KNeighborsClassifier(3)
#clf = DecisionTreeClassifier(max_depth=5)
#clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
#clf = MLPClassifier()
clf = GaussianNB()

In [12]:
clf.fit(X_train, y_train) 

GaussianNB(priors=None, var_smoothing=1e-09)

In [13]:
y_pred = clf.predict(X_test)

In [14]:
pred_labels = le.inverse_transform(y_pred)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
print("Accuracy = {0:.2f}".format(accuracy))
print("Precision = {0:.2f}".format(precision))
print("Recall = {0:.2f}".format(recall))

Accuracy = 0.37
Precision = 0.46
Recall = 0.37


In [17]:
text = "ओं नमः श्रीशारदागणपतिगुरुभ्यः श्री शारदा गणपति गुरुभ्यः"

In [18]:
tokens = text.split()

In [19]:
test_data = []
for i in range(len(tokens)):
    test_data.append(model_wv.wv[tokens[i]])

In [20]:
yp = clf.predict(test_data)

In [21]:
le.inverse_transform(yp)

array(['ADJ', 'NOUN', 'ADJ', 'ADJ', 'ADJ', 'NOUN', 'NOUN'], dtype='<U23')

In [22]:
from pyfasttext import FastText

In [23]:
model_ft = FastText()
model_ft.skipgram(input='POS_Sanskrit.txt', output='model_ft', epoch=10, lr=0.1, dim=300)

In [24]:
X = []
for i in range(len(words[0])):
    X.append(model_ft[words[0][i]])

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [26]:
print(np.shape(X_train),np.shape(y_train))
print(np.shape(X_test), np.shape(y_test))

(29997, 300) (29997,)
(12856, 300) (12856,)


In [27]:
#clf = SVC()
clf = KNeighborsClassifier(3)
#clf = DecisionTreeClassifier(max_depth=5)
#clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
#clf = MLPClassifier()
#clf = GaussianNB()

clf.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [28]:
y_pred = clf.predict(X_test)

In [29]:
pred_labels = le.inverse_transform(y_pred)

In [30]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
print("Accuracy = {0:.2f}".format(accuracy))
print("Precision = {0:.2f}".format(precision))
print("Recall = {0:.2f}".format(recall))

Accuracy = 0.69
Precision = 0.68
Recall = 0.69


In [34]:
text = "ओं नमः श्रीशारदागणपतिगुरुभ्यः श्री शारदा गणपति गुरुभ्य"

In [35]:
tokens = text.split()

In [36]:
test_data = []
for i in range(len(tokens)):
    test_data.append(model_ft[tokens[i]])

In [37]:
yp = clf.predict(test_data)

In [38]:
le.inverse_transform(yp)

array(['', 'NOUN', '_', 'ADJ', 'X', 'VERB', 'NOUN'], dtype='<U23')

In [39]:
import numpy as np
from keras.models import Sequential
from keras import optimizers
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from keras import backend as K
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [41]:
path = 'POS_Sanskrit.txt'

In [42]:
def get_data(path):
    texts = []
    labels = []
    read_file = open(path, 'r')
    for line in read_file:
        line = line.replace('\n','')
        if line == 'newline':
            pass
        else:
            items = line.split('\t')
            texts.append(items[0])
            labels.append(items[1])
    read_file.close()

    return texts, labels

In [43]:
words, tags = get_data(path)
words = [words]
wvmodel = Word2Vec(words, size=300, window=1, min_count=1)

In [44]:
X = []
for i in range(len(words[0])):
    X.append(wvmodel.wv[words[0][i]])


m, n = np.shape(X)
X = np.reshape(X, (m,n,1))

In [45]:
label_encoder = LabelEncoder()
label_encoder.fit(tags)
y = label_encoder.transform(tags)
y = to_categorical(y, len(set(y)))

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [47]:
batch_size = 32

In [50]:
model = Sequential()
model.add(LSTM(32, activation='tanh', input_shape = X_train.shape[1:]))
model.add(Dense(26, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_2 (Dense)              (None, 26)                858       
Total params: 5,210
Trainable params: 5,210
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train,  batch_size = batch_size, epochs = 10, validation_data = (X_test, y_test))

Train on 29997 samples, validate on 12856 samples
Epoch 1/10
29997/29997 [==============================] - 92s 3ms/step - loss: 2.1615 - acc: 0.3001 - val_loss: 2.0805 - val_acc: 0.2986
Epoch 2/10
29997/29997 [==============================] - 94s 3ms/step - loss: 2.0848 - acc: 0.3007 - val_loss: 2.0803 - val_acc: 0.2986
Epoch 3/10
29997/29997 [==============================] - 97s 3ms/step - loss: 2.0849 - acc: 0.3007 - val_loss: 2.0780 - val_acc: 0.2986
Epoch 4/10
29997/29997 [==============================] - 97s 3ms/step - loss: 2.0843 - acc: 0.3007 - val_loss: 2.0816 - val_acc: 0.2986
Epoch 5/10
29997/29997 [==============================] - 94s 3ms/step - loss: 2.0845 - acc: 0.3007 - val_loss: 2.0821 - val_acc: 0.2986
Epoch 6/10
29997/29997 [==============================] - 92s 3ms/step - loss: 2.0844 - acc: 0.3007 - val_loss: 2.0809 - val_acc: 0.2986
Epoch 7/10
29997/29997 [==============================] - 93s 3ms/step - loss: 2.0842 - acc: 0.3007 - val_loss: 2.0800 - val_acc

In [ ]:
score = model.evaluate(X_test, y_test)
print("Accuracy = ",score[1]*100)

In [ ]:
y_pred = model.predict_classes(X_test)
pred_labels = le.inverse_transform(y_pred)
print(pred_labels)

In [ ]:
text = "ओं नमः श्रीशारदागणपतिगुरुभ्यः श्री शारदा गणपति गुरुभ्य"
tokens = text.split()

In [ ]:
test_data = []
for i in range(len(tokens)):
    test_data.append(wvmodel[tokens[i]])

In [ ]:
mt, nt = np.shape(test_data)
test_data = np.reshape(test_data, (mt, nt,1))

In [ ]:
y_pred = model.predict_classes(test_data)
pred_labels = le.inverse_transform(y_pred)
print(pred_labels)